<img src="http://akhavanpour.ir/notebook/images/srttu.gif" alt="SRTTU" style="width: 150px;"/>

[![Azure Notebooks](https://notebooks.azure.com/launch.png)](https://notebooks.azure.com/import/gh/Alireza-Akhavan/class.vision)


# Book Recommendation System

# <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">سیستم توصیه گر کتاب</div>

<div style="direction:rtl;text-align:right;font-family:Tahoma">
کدها برگرفته از مستندات بلاگ :</div>

[https://towardsdatascience.com/building-a-book-recommendation-system-using-keras-1fba34180699
](https://towardsdatascience.com/building-a-book-recommendation-system-using-keras-1fba34180699)



<div style="direction:rtl;text-align:right;font-family:Tahoma">
و کدهای موجود در مخزن کد:</div>

[https://github.com/TannerGilbert/Articles](https://github.com/TannerGilbert/Articles)


<div style="direction:rtl;text-align:right;font-family:Tahoma">
است<br>
</div>


# <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">مجموعه داده</div>

<div style="direction:rtl;text-align:right;font-family:Tahoma">
کدها برگرفته از مستندات بلاگ :</div>

<div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">
مجموعه داده‌ی استفاده شده در این نوت بوک   goodbooks-10k سایت Kaggle است.
</div>

[https://www.kaggle.com/zygmunt/goodbooks-10k](https://www.kaggle.com/zygmunt/goodbooks-10k)


<div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">
ویژگی های موجود در دیتاست:</div>

        -book_id
        -user_id
        -rating

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model


In [2]:
dataset = pd.read_csv('https://github.com/Alireza-Akhavan/datasets_and_models/raw/main/ratings.csv')

In [3]:
dataset.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [4]:
dataset.shape

(981756, 3)

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42) #### دیفالت ترین تست اسپلیت اینه که شافل می کنه

In [6]:
train.head() #### الان شافل شده میده

,book_id,user_id,rating
341848,3423,4608,2
964349,9811,36373,5
645459,6485,2957,4
74960,750,42400,3
358670,3591,36886,5


In [7]:
test.head() ### شافل شده میده

,book_id,user_id,rating
646451,6495,19643,5
614851,6175,8563,4
974393,9920,52110,3
21471,215,33864,5
272540,2728,16587,3


In [8]:
n_users = len(dataset.user_id.unique())
n_users

53424

In [9]:
n_books = len(dataset.book_id.unique())
n_books

10000

In [10]:
len(dataset)

981756

In [11]:
# اگر میخواست جدول کلا پر باشه، باید 53424*10000 رای مداشتم ولی الان فقط 981756 تا دارم
981756/(10000*53424)*100

### یعنی فقط 18 درصد اون ماتریس پره و به شدت اسپارس هست و خیلیاش باید پر شده، پس باید یه فریم ورک دیپ لرنینگی بیاد حلش کنه

0.18376684636118598

<div style="direction:rtl;text-align:right;font-family:Tahoma">
مدلی که خواهیم داشت از اجزای اصلی زیر تشکیل خواهد شد</div>

    -Input: Input for both books and users
    -Embedding Layers: Embeddings for books and users
    -Dot: combines embeddings using a dot product

در این شبکه، شما به ازای هر ایدی بوک یه امبدینگ 5 تایی می سازی و به ازای هر یوزر ایدی هم یه امبدینگ، که شبکه میاد هر امبدینگ رو فلتن میکنه و ضرب داخلی بین یوزر و کتاب رو خواهیم دفمفثد###mm

In [12]:
#### شبکه رو با فانکشنال ای پی آی ساخته
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input) ### به اضافه یک به خاطر اینکه امبدینگ از 0 شروع میشه و یوززها و کتابها از یک
book_vec = Flatten(name="Flatten-Books")(book_embedding)

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

prod = Dot(name="Dot-Product", axes=1)([book_vec, user_vec])
model = Model([user_input, book_input], prod)
model.compile('adam', 'mean_squared_error')

In [13]:
model.summary() #### اینجا به ازای هر مینی بچ که اینجا ان none هست، ما یه یه یوزر آی  دی برای کتاب داریم مثلا 32 تا یوزر آی دی کتاب داریم که در یک هستند چون هر کتاب یه کلمه هست و بعد که امنبد میشه در 5 ضرب میشه، مثل تکست که مثلا 32 تا جمله داشتیم در تعداد توکن ها در عدد امبدینگ

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Book-Input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 User-Input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 Book-Embedding (Embedding)  (None, 1, 5)                 50005     ['Book-Input[0][0]']          
                                                                                                  
 User-Embedding (Embedding)  (None, 1, 5)                 267125    ['User-Input[0][0]']          
                                                                                              

In [14]:
model.evaluate([test.user_id, test.book_id], test.rating) ### هنوز ترین نکرده و قاعدتا خطا بالاست

6136/6136 [==============================] - 17s 3ms/step - loss: 15.8283


15.82834529876709

In [16]:
predictions = model.predict([test.user_id.head(10), test.book_id.head(10)]) ### هنوز ترین نشده و قاعدتا اشتباه تخمین میزنه

for i in range(0,10):
   print(predictions[i], test.rating.iloc[i])

1/1 [==============================] - 0s 19ms/step
[-0.00021684] 5
[-0.00104155] 4
[-0.0023749] 3
[0.00139343] 5
[0.00356162] 3
[-0.00220543] 3
[-0.00266467] 3
[-0.00258496] 4
[0.00152363] 3
[0.00172637] 5


In [ ]:
from keras.models import load_model
import os

if os.path.exists('regression_model.h5'):
    model = load_model('regression_model.h5')
else:
    history = model.fit([train.user_id, train.book_id], train.rating, epochs=15, verbose=1024)
    model.save('regression_model.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

Epoch 1/15
Epoch 2/15


In [ ]:
model.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 16s 3ms/step - loss: 0.9603


0.9602550268173218

In [ ]:
predictions = model.predict([test.user_id.head(10), test.book_id.head(10)])

[print(predictions[i], test.rating.iloc[i]) for i in range(0,10)]

1/1 [==============================] - 0s 21ms/step
[5.2514153] 5
[3.7159991] 4
[3.7052584] 3
[3.8982012] 5
[3.5516946] 3
[3.5172405] 3
[2.9716544] 3
[4.7415504] 4
[4.4628468] 3
[3.9871783] 5


[None, None, None, None, None, None, None, None, None, None]